In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--window-size=1920,1200")

# the line below if for preventing window closing
# options.headless = True

DRIVER_PATH = 'chromedriver'
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

# init
books = {"Title": [],
         "Price": [],
         "Rating": [],
         "Availability": [],
         "Image URL": []}

# page loading for each page
driver.get("https://books.toscrape.com/catalogue/page-1.html")

# data scrapping for each item of the page
title = driver.find_element(By.XPATH, '/html/body/div/div/div/div/section/div[2]/ol/li[1]/article/h3/a')
price = driver.find_element(By.XPATH, '/html/body/div/div/div/div/section/div[2]/ol/li[1]/article/div[2]/p[1]')
stock = driver.find_element(By.XPATH, '/html/body/div/div/div/div/section/div[2]/ol/li[1]/article/div[2]/p[2]')
stars = driver.find_element(By.XPATH, '/html/body/div/div/div/div/section/div[2]/ol/li[1]/article/p')
image = driver.find_element(By.XPATH, '/html/body/div/div/div/div/section/div[2]/ol/li[1]/article/div[1]/a/img')

# putting values in the dictionary
books["Title"].append(title.get_attribute("title"))
books["Price"].append(price.text)
books["Rating"].append(stars.get_attribute("class"))
books["Availability"].append(stock.text)
books["Image URL"].append(image.get_attribute("src"))

# dict -> dataframe
df_books = pd.DataFrame(books)
print(df_books)
df_books.to_csv('books.csv')

driver.quit()

                  Title   Price             Rating Availability  \
0  A Light in the Attic  £51.77  star-rating Three     In stock   

                                           Image URL  
0  https://books.toscrape.com/media/cache/2c/da/2...  
